In [37]:
from bs4 import BeautifulSoup
import glob
import os

courses_directory = 'nbgrader'
#course = ""
#assignment = ""

#feedback = os.path.join(courses_directory, course, 'feedback', '*', assignment, '*.html')
#feedback

def getResult():
    return result

def setPath(c, a):
    course = c
    assignment = a
    feedback = os.path.join(courses_directory, course, 'feedback', '*', assignment, '*.html')
    return feedback
    #print(feedback)

def fetch(feedback):
    feedback_files = glob.glob(feedback)
    #feedback_files
    if (not feedback_files):
        print("no feedback to summarize.")
        return
    
    result = ""
    for f in feedback_files:
        
        normalized_path = os.path.normpath(f)
        print(normalized_path)
        # Split the normalized path into its components
        parts = normalized_path.split(os.sep)  # os.sep is the appropriate separator
        index = parts.index('feedback')
        studentId = parts[index + 1]
        
        with open(normalized_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        soup = BeautifulSoup(html_content, 'html.parser')
        
        ph = soup.find('div', class_="panel-heading")
        overall = str(ph.find('h4')).split("Score:")[1].split(")")[0].strip()
        result += (studentId + "\n")
        result += (f"Overall Score: {overall}\n")
    
        qs = ph.find('ol').find_all('li')
        count = 1
        for q in qs:
            href = str(q)[str(q).find('#')+1:str(q).find('">')]
            t = BeautifulSoup(str(q), 'html.parser').get_text()
            
            if (t == "Comment"):
                anchor = soup.find('a', attrs={'name': href})
                div = anchor.find_parent('div')
                comment = BeautifulSoup(str(div.find('div', class_="panel-footer").find('div').get_text()), 'html.parser')
                result += (comment.get_text().strip() + "\n")
            else:
                result += (str(count) + ". " + t + "\n")
                count += 1   
            
        result += ("--------------------------\n")
    return result

In [52]:
import ipywidgets as widgets
from IPython.display import display
import os

courses = [name for name in os.listdir(courses_directory) if os.path.isdir(os.path.join(courses_directory, name))]
courses.insert(0, "---SELECT COURSE---")

course = widgets.Dropdown(
    options=courses,
    description='Course:',
    disabled=False
)

assignment = widgets.Dropdown(
    options=[],
    description='Assignment:',
    disabled=False
)

button = widgets.Button(
    description='Submit',
    layout=widgets.Layout(margin='0px 0px 0px 90px'), 
    disabled=True
)

textbox = widgets.Textarea(
    description='Output:',
    layout=widgets.Layout(width='50%', height='300px'),  # Set width and height
    disabled=True
)

def course_change(change):
    selected_course = change['new']
    if (selected_course == "---SELECT COURSE---"):
        assignment.options = []  # Clear options if no assignments exist
        assignment.value = None
    else:
        assignment_path = os.path.join(courses_directory, selected_course, 'source')
        if os.path.exists(assignment_path):
            assignments = [name for name in os.listdir(assignment_path) if os.path.isdir(os.path.join(assignment_path, name))]
            assignment.options = assignments  # Update assignment options
            assignment.value = assignments[0]
        else:
            assignment.options = []  # Clear options if no assignments exist
            assignment.value = None

def assignment_change(change):
    if assignment.value is not None:
        button.disabled = False
    else:
        button.disabled = True

def on_button_clicked(b):
    f = setPath(course.value, assignment.value)
    t = fetch(f)
    if (t):
        textbox.value = "Executing..."
        button.disabled = True
        %run -i summarize.ipynb
        result = summarize(t)
        textbox.value = result
        button.disabled = False

course.observe(course_change, names='value')
assignment.observe(assignment_change, names='value')
button.on_click(on_button_clicked)

display(course, assignment, button, textbox)

Dropdown(description='Course:', options=('---SELECT COURSE---', 'course', 'newcourse'), value='---SELECT COURS…

Dropdown(description='Assignment:', options=(), value=None)

Button(description='Submit', disabled=True, layout=Layout(margin='0px 0px 0px 90px'), style=ButtonStyle())

Textarea(value='', description='Output:', disabled=True, layout=Layout(height='300px', width='50%'))